# Store pdf on the acceleration only and stats in zarr

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl
from matplotlib.ticker import FormatStrFormatter

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import m2lib22.box as box
import m2lib22.aviso as aviso
import m2lib22.cstes as cstes
import m2lib22.diagnosis as diag
import m2lib22.erastar as eras
import m2lib22.stress_to_windterm as stw

from m2lib22.cstes import labels, zarr_dir

In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=10, processes=10, walltime='04:00:00')
    w = cluster.scale(jobs=4)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.70:8787/status,
Dashboard: http://10.148.1.70:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.70:36172,Workers: 0
Dashboard: http://10.148.1.70:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## IMPORT DATA, build dataset
### FOR ALL DATA

In [3]:
# no PEACHY data
labels=labels[:2]

# only erastar
#list_wd_srce_suffix = ['es']
#_stress_var = [v for v in diag._stress_var if 'es' in v]

DS={}
for l in labels:
    #OPEN DATA FILES
    ds_data = xr.open_zarr(zarr_dir+'/'+l+'.zarr')
    ds_stress= xr.open_zarr(zarr_dir+'/erastar/erastar_'+l+'.zarr')# TO CHANGE ONCE GOOD FILES GENERATED
    ds_aviso = xr.open_zarr(zarr_dir+'/aviso_'+l+'.zarr')    
    #
    ds = xr.merge(diag.datasets_for_pdfs(ds_data, ds_aviso, ds_stress, ds_corr, sum_ = True, except_ = True,
                     ))
    ds['aviso_alti_matchup_err_sla']=ds_aviso['aviso_alti_matchup_err_sla']
    ds['aviso_drifter_matchup_err_sla']=ds_aviso['aviso_traj_err_sla'].isel(site_obs = ds_data.__site_matchup_indice.compute())
    #dt<0.5h, dl<1e5m,
    ds= ds.where(ds.alti___time_difference<=1800, drop=True)
    ds= ds.where(ds.alti___distance<=1e5, drop=True)
    DS[l]=ds.persist()
    print(l)

gps_SASSA_SARAL_2018
gps_SASSA_Sentinel_2018


In [5]:
DS[labels[0]]

<xarray.Dataset>
Dimensions:                                                    (obs: 66946,
                                                                box_x: 80,
                                                                box_y: 40,
                                                                id_comb: 72)
Coordinates: (12/13)
    alti___distance                                            (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    alti___time_difference                                     (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
  * box_x                                                      (box_x) float64 ...
  * box_y                                                      (box_y) float64 ...
    drifter_lat                                                (obs) float64 dask.array<chunksize=(66946,), meta=np.ndarray>
    drifter_lon                                                (obs) float64 dask.array<chunksize=(66946,), meta=np.ndarray>
    ...                                                         ...
    drifter_x                                                  (obs) float64 dask.array<chunksize=(66946,), meta=np.ndarray>
    drifter_y                                                  (obs) float64 dask.array<chunksize=(66946,), meta=np.ndarray>
    lat                                                        (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    lon                                                        (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    time                                                       (obs) datetime64[ns] dask.array<chunksize=(33432,), meta=np.ndarray>
  * id_comb                                                    (id_comb) <U44 ...
Dimensions without coordinates: obs
Data variables: (12/355)
    alti_g_grad_x                                              (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    alti_denoised_g_grad_x                                     (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    drifter_acc_x                                              (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    drifter_acc_y                                              (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    drifter_coriolis_x                                         (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    drifter_coriolis_y                                         (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    ...                                                         ...
    sum_aviso_es_cstrio_z0_drifter_y                           (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    sum_aviso_e5_cstrio_z0_drifter_y                           (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    sum_aviso_es_cstrio_z15_drifter_y                          (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    sum_aviso_e5_cstrio_z15_drifter_y                          (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    aviso_alti_matchup_err_sla                                 (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
    aviso_drifter_matchup_err_sla                              (obs) float64 dask.array<chunksize=(415,), meta=np.ndarray>
Attributes:
    __id:                   SASSA-L3-SLA-SARAL__gps
    __time_coverage_end:    2019-01-01T00:00:00
    __time_coverage_start:  2018-01-01T00:00:00

In [6]:
ds = xr.concat([DS[key] for key in DS], dim='obs').chunk({'obs':500})

In [7]:
ds

<xarray.Dataset>
Dimensions:                                                    (obs: 133258,
                                                                box_x: 80,
                                                                box_y: 40,
                                                                id_comb: 72)
Coordinates: (12/13)
    alti___distance                                            (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti___time_difference                                     (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
  * box_x                                                      (box_x) float64 ...
  * box_y                                                      (box_y) float64 ...
    drifter_lat                                                (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_lon                                                (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    ...                                                         ...
    drifter_x                                                  (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_y                                                  (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    lat                                                        (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    lon                                                        (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    time                                                       (obs) datetime64[ns] dask.array<chunksize=(500,), meta=np.ndarray>
  * id_comb                                                    (id_comb) <U44 ...
Dimensions without coordinates: obs
Data variables: (12/355)
    alti_g_grad_x                                              (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    alti_denoised_g_grad_x                                     (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_acc_x                                              (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_acc_y                                              (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_coriolis_x                                         (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_coriolis_y                                         (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    ...                                                         ...
    sum_aviso_es_cstrio_z0_drifter_y                           (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    sum_aviso_e5_cstrio_z0_drifter_y                           (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    sum_aviso_es_cstrio_z15_drifter_y                          (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    sum_aviso_e5_cstrio_z15_drifter_y                          (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    aviso_alti_matchup_err_sla                                 (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    aviso_drifter_matchup_err_sla                              (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
Attributes:
    __id:                   SASSA-L3-SLA-SARAL__gps
    __time_coverage_end:    2019-01-01T00:00:00
    __time_coverage_start:  2018-01-01T00:00:00

In [10]:
zarr = os.path.join(zarr_dir, f"ds_matchup_gps_05_100.zarr")
ds.isel(obs=slice(60000,133258)).to_zarr(zarr, mode="a", append_dim='obs')  
print(f"ds storred in {zarr}")

ds storred in /home1/datawork/mdemol/m2/ds_matchup_gps_05_100.zarr


In [11]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
